In [ ]:
import pandas as pd
import aiohttp
import asyncio
import base64
import time
from itertools import cycle
from tqdm import tqdm
import nest_asyncio

# Allow running asyncio in Jupyter Notebook
nest_asyncio.apply()

# Load dataset
df = pd.read_csv("github_repos_filtered_language.csv")

# Keywords to search in README
keywords = {"aws", "azure", "terraform", "google", "gcp", "docker", "amazon",
            "devops", "kubernetes", "microsoft", "jenkins", "cloud"}

# GitHub API Tokens (Rotating 6 tokens)
TOKENS = [
    "placeholder"
]

# Global rate limit tracking
RATE_LIMITS = {token: 5000 for token in TOKENS}
RATE_RESETS = {token: time.time() for token in TOKENS}

# Function to get next available token (waits if all tokens are exhausted)
async def get_next_token():
    first_wait_print = True  # Flag to track first expired message

    while True:
        for token in TOKENS:
            if RATE_LIMITS[token] > 0 or time.time() > RATE_RESETS[token]:  
                return token

        # All tokens exhausted, calculate minimum reset time
        min_reset_time = min(RATE_RESETS.values()) - time.time()
        if min_reset_time > 0:
            if first_wait_print:
                print(f"⏳ All tokens exhausted. Waiting {min_reset_time / 60:.1f} minutes until reset...")
                first_wait_print = False  # Ensure we don't spam the message

        # Print wait time only every 5 minutes
        while min_reset_time > 0:
            wait_time = min(300, max(10, int(min_reset_time)))  # Max wait = 5 min, Min wait = 10 sec
            print(f"⏳ Tokens exhausted. Next reset in {min_reset_time / 60:.1f} minutes...")
            await asyncio.sleep(wait_time)
            min_reset_time = min(RATE_RESETS.values()) - time.time()

# Function to check rate limit for a token
async def check_rate_limit(session, token):
    global RATE_LIMITS, RATE_RESETS

    url = "https://api.github.com/rate_limit"
    headers = {"Authorization": f"token {token}", "Accept": "application/vnd.github.v3+json"}

    async with session.get(url, headers=headers) as response:
        if response.status != 200:
            print(f"Error fetching rate limit for token {token[:10]}...: {response.status}")
            return  
        
        data = await response.json()

        if "rate" in data:
            RATE_LIMITS[token] = data["rate"]["remaining"]
            RATE_RESETS[token] = data["rate"]["reset"]

            reset_time = RATE_RESETS[token] - time.time()
            print(f"API Token {token[:10]}...: {RATE_LIMITS[token]} remaining, resets in {reset_time:.2f} seconds")
        else:
            print(f"Unexpected response for token {token[:10]}: {data}")

# Function to fetch README content
async def fetch_readme(session, owner, repo):
    possible_filenames = ["README.md", "README.txt", "README"]
    
    for filename in possible_filenames:
        token = await get_next_token()  # Get valid token
        url = f"https://api.github.com/repos/{owner}/{repo}/contents/{filename}"
        headers = {"Authorization": f"token {token}", "Accept": "application/vnd.github.v3+json"}

        async with session.get(url, headers=headers) as response:
            if response.status == 403:  # Rate limit hit
                await check_rate_limit(session, token)
                return None  # Return None instead of recursion

            if response.status == 200:
                data = await response.json()

                #Ensure response is a dictionary before accessing "content"
                if isinstance(data, dict) and "content" in data:
                    content = data["content"]
                    return base64.b64decode(content).decode("utf-8", errors="ignore").lower()
                
                print(f"Unexpected API response for {owner}/{repo}: {data}")
                return None

    return None  # No README found

# Function to process repositories asynchronously
async def process_repos(batch_size=100):
    async with aiohttp.ClientSession() as session:
        repo_list = df["full_name"].tolist()

        for batch_start in range(0, len(repo_list), batch_size):
            batch_end = min(batch_start + batch_size, len(repo_list))
            print(f"\nProcessing batch {batch_start + 1} to {batch_end} of {len(repo_list)}...")

            tasks = []
            for idx in range(batch_start, batch_end):
                owner, repo = repo_list[idx].split("/")
                tasks.append(fetch_readme(session, owner, repo))

            results = await asyncio.gather(*tasks)

            # Process results
            for idx, readme_text in tqdm(enumerate(results, start=batch_start), total=batch_size, desc="🔄 Updating Descriptions"):
                if readme_text:
                    matched_keywords = {kw for kw in keywords if kw in readme_text}
                    if matched_keywords:
                        original_desc = df.at[idx, "description"] if pd.notna(df.at[idx, "description"]) else ""
                        df.at[idx, "description"] = (original_desc + " " + " ".join(matched_keywords)).strip()

            # Save after each batch
            df.to_csv("github_repos_filled_descriptions.csv", index=False)
            print(f"Saved progress at {batch_end}/{len(repo_list)}")

            # Check API rate limit after each batch
            for token in TOKENS:
                await check_rate_limit(session, token)

# Run async function
await process_repos()



📄 Processing batch 1 to 100 of 121557...


🔄 Updating Descriptions: 100%|████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 20676.87it/s]


📁 Saved progress at 100/121557
🚦 API Token ghp_Vefb3d...: 4876 remaining, resets in 3596.44 seconds
🚦 API Token ghp_TpKN4h...: 5000 remaining, resets in 3597.31 seconds
🚦 API Token ghp_Iw9IkC...: 5000 remaining, resets in 3597.19 seconds
🚦 API Token ghp_qtCGV8...: 5000 remaining, resets in 3597.06 seconds
🚦 API Token ghp_griLO9...: 0 remaining, resets in 12.94 seconds
🚦 API Token ghp_2hLFOd...: 0 remaining, resets in 105.81 seconds

📄 Processing batch 101 to 200 of 121557...


🔄 Updating Descriptions: 100%|████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 21571.20it/s]


📁 Saved progress at 200/121557
🚦 API Token ghp_Vefb3d...: 4712 remaining, resets in 3594.58 seconds
🚦 API Token ghp_TpKN4h...: 5000 remaining, resets in 3597.46 seconds
🚦 API Token ghp_Iw9IkC...: 5000 remaining, resets in 3597.33 seconds
🚦 API Token ghp_qtCGV8...: 5000 remaining, resets in 3597.21 seconds
🚦 API Token ghp_griLO9...: 0 remaining, resets in 11.10 seconds
🚦 API Token ghp_2hLFOd...: 0 remaining, resets in 103.97 seconds

📄 Processing batch 201 to 300 of 121557...


...

📁 Saved progress at 121400/121557
🚦 API Token ghp_Vefb3d...: 0 remaining, resets in 3137.20 seconds
🚦 API Token ghp_TpKN4h...: 0 remaining, resets in 3226.09 seconds
🚦 API Token ghp_Iw9IkC...: 0 remaining, resets in 3314.97 seconds
🚦 API Token ghp_qtCGV8...: 0 remaining, resets in 3407.85 seconds
🚦 API Token ghp_griLO9...: 0 remaining, resets in 3497.70 seconds
🚦 API Token ghp_2hLFOd...: 4382 remaining, resets in 3589.57 seconds

📄 Processing batch 121401 to 121500 of 121557...


🔄 Updating Descriptions: 100%|████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 11925.46it/s]


📁 Saved progress at 121500/121557
🚦 API Token ghp_Vefb3d...: 0 remaining, resets in 3134.53 seconds
🚦 API Token ghp_TpKN4h...: 0 remaining, resets in 3223.41 seconds
🚦 API Token ghp_Iw9IkC...: 0 remaining, resets in 3312.30 seconds
🚦 API Token ghp_qtCGV8...: 0 remaining, resets in 3405.15 seconds
🚦 API Token ghp_griLO9...: 0 remaining, resets in 3494.97 seconds
🚦 API Token ghp_2hLFOd...: 4236 remaining, resets in 3586.85 seconds

📄 Processing batch 121501 to 121557 of 121557...


🔄 Updating Descriptions:  57%|█████████████████████████████████████                            | 57/100 [00:00<00:00, 12963.63it/s]


📁 Saved progress at 121557/121557
🚦 API Token ghp_Vefb3d...: 0 remaining, resets in 3131.82 seconds
🚦 API Token ghp_TpKN4h...: 0 remaining, resets in 3220.70 seconds
🚦 API Token ghp_Iw9IkC...: 0 remaining, resets in 3309.58 seconds
🚦 API Token ghp_qtCGV8...: 0 remaining, resets in 3402.46 seconds
🚦 API Token ghp_griLO9...: 0 remaining, resets in 3492.35 seconds
🚦 API Token ghp_2hLFOd...: 4141 remaining, resets in 3584.22 seconds
